In [5]:
from clutter_filter import GroundClutterFilter
import numpy as np
import glob, os
import radarkit

In [6]:
rkid = radarkit.open('/home/vitorgoede/radar_data/Raxpol/20250605/RAXPOL-20250605-223617.946.rkc')

In [7]:
out = rkid.read()

Estimated pulseCount = 79,989   gateCount = 1,245   downSampledGateCount = 1,245


100%|████████████████████████████████████████████████████████████████████████▉|00:02<00:00


In [8]:
ciq = out['ciq']

lamb = rkid.desc.wavelength
prt = radarkit.read_RKFloat_array(rkid.configs.contents.prt,1)
c = 3e8
nyq_val = lamb/(4*prt)
C = np.abs(radarkit.read_RKComplex_array(rkid.configs.contents.systemZCal, 1)[0])
l = np.abs(radarkit.read_RKComplex_array(rkid.configs.contents.systemZCal, 2)[1])

count = 55
rayCount = 1440
gateCount = ciq.shape[2]
n = rayCount * count
#n = 55011
X_h = np.reshape(ciq[:n, 0, :], (rayCount, count, -1))
X_h = np.transpose(X_h, (2, 0, 1))

X_v = np.reshape(ciq[:n, 1, :], (rayCount, count, -1))
X_v = np.transpose(X_v, (2, 0, 1))

In [9]:
# set some imporatnt radar parameter
wavelength_scalar = lamb
prt_scalar = prt[0]
num_samples_actual = X_h.shape[2]

#filter the I/Q time series for the horizontal and vertical polarizations
filter_inst = GroundClutterFilter(
    wavelength=lamb,
    scan_rate=1/(count * prt[0]),
    prt=prt[0],
    num_samples=count
)

print("\n--- Filtering X_h (Horizontal Polarization) ---")
filtered_i_h, filtered_q_h, poly_order_map_h, clutter_i_trend_map_h, clutter_q_trend_map_h, interpolated_spectrum_map_h = \
    filter_inst.filter_iq_data(X_h.real, X_h.imag, cnr_db_map=None, apply_interpolation=True)

filtered_X_h_optimized = filtered_i_h + 1j * filtered_q_h

del filtered_i_h, filtered_q_h, poly_order_map_h, clutter_i_trend_map_h, clutter_q_trend_map_h, interpolated_spectrum_map_h

print("\n--- Filtering X_v (Vertical Polarization) ---")
filtered_i_v, filtered_q_v, poly_order_map_v, clutter_i_trend_map_v, clutter_q_trend_map_v, interpolated_spectrum_map_v = \
    filter_inst.filter_iq_data(X_v.real, X_v.imag, cnr_db_map=None, apply_interpolation=True)

filtered_X_v_optimized = filtered_i_v + 1j * filtered_q_v

del filtered_i_v, filtered_q_v, poly_order_map_v, clutter_i_trend_map_v, clutter_q_trend_map_v, interpolated_spectrum_map_v

print("\nOptimization complete. Filtered data is in filtered_X_h_optimized and filtered_X_v_optimized.")

Filter initialized with:
  Wavelength: 0.0308 m
  Scan Rate: 72.72726927291279 deg/s
  PRT: 0.0002500000118743628 s
  Number of Samples per gate: 55
  Nyquist Velocity: 30.80 m/s

--- Filtering X_h (Horizontal Polarization) ---
Processing 1792800 cells (1245 range gates, 1440 azimuthal positions)...
Estimating CNR for all cells...
  _estimate_clutter_power_from_polyfit: i_series.shape = (1792800, 55)
  _estimate_clutter_power_from_polyfit: q_series.shape = (1792800, 55)
  _estimate_clutter_power_from_polyfit: t.shape = (55,)
  _estimate_clutter_power_from_polyfit: i_series_T.shape = (55, 1792800)
  _estimate_clutter_power_from_polyfit: coeffs_i.shape = (3, 1792800)
  _estimate_clutter_power_from_polyfit: clutter_i.shape = (1792800, 55)
  _estimate_clutter_power_from_polyfit: clutter_q.shape = (1792800, 55)
  _estimate_clutter_power_from_polyfit: clutter_power_i.shape = (1792800,)
  _estimate_clutter_power_from_polyfit: total_clutter_power.shape = (1792800,)
Determining polynomial order

Batch Gaussian Interpolation:   0%|          | 0/317361 [00:00<?, ?it/s]


--- Filtering X_v (Vertical Polarization) ---
Processing 1792800 cells (1245 range gates, 1440 azimuthal positions)...
Estimating CNR for all cells...
  _estimate_clutter_power_from_polyfit: i_series.shape = (1792800, 55)
  _estimate_clutter_power_from_polyfit: q_series.shape = (1792800, 55)
  _estimate_clutter_power_from_polyfit: t.shape = (55,)
  _estimate_clutter_power_from_polyfit: i_series_T.shape = (55, 1792800)
  _estimate_clutter_power_from_polyfit: coeffs_i.shape = (3, 1792800)
  _estimate_clutter_power_from_polyfit: clutter_i.shape = (1792800, 55)
  _estimate_clutter_power_from_polyfit: clutter_q.shape = (1792800, 55)
  _estimate_clutter_power_from_polyfit: clutter_power_i.shape = (1792800,)
  _estimate_clutter_power_from_polyfit: total_clutter_power.shape = (1792800,)
Determining polynomial orders...
Performing regression filtering on I series...
  _perform_regression_filter: selected_series.shape = (272205, 55)
  _perform_regression_filter: clutter_trend.shape = (272205, 5

Batch Gaussian Interpolation:   0%|          | 0/332825 [00:00<?, ?it/s]


Optimization complete. Filtered data is in filtered_X_h_optimized and filtered_X_v_optimized.
